## Exercises

The end result of this exercise should be a file named prepare.py that defines the requested functions.

In this exercise we will be defining some functions to prepare textual data. These functions should apply equally well to both the codeup blog articles and the news articles that were previously acquired.

Define a function named basic_clean. It should take in a string and apply some basic text cleaning to it:

- Lowercase everything
- Normalize unicode characters
- Replace anything that is not a letter, number, whitespace or a single quote.

Define a function named tokenize. It should take in a string and tokenize all the words in the string.

Define a function named stem. It should accept some text and return the text after applying stemming to all the words.

Define a function named lemmatize. It should accept some text and return the text after applying lemmatization to each word.

Define a function named remove_stopwords. It should accept some text and return the text after removing all the stopwords.

This function should define two optional parameters, extra_words and exclude_words. These parameters should define any additional stop words to include, and any words that we don't want to remove.

Use your data from the acquire to produce a dataframe of the news articles. Name the dataframe news_df.

Make another dataframe for the Codeup blog posts. Name the dataframe codeup_df.

For each dataframe, produce the following columns:

- title to hold the title
- original to hold the original article/post content
- clean to hold the normalized and tokenized original with the stopwords removed.
- stemmed to hold the stemmed version of the cleaned data.
- lemmatized to hold the lemmatized version of the cleaned data.

Ask yourself:

- If your corpus is 493KB, would you prefer to use stemmed or lemmatized text?
- If your corpus is 25MB, would you prefer to use stemmed or lemmatized text?
- If your corpus is 200TB of text and you're charged by the megabyte for your hosted computational resources, would you prefer to use stemmed or lemmatized text?

In [1]:
import pandas as pd
import numpy as np
import re
import unicodedata
import nltk
from nltk.corpus import stopwords

In [2]:
def basic_clean(string):
    '''This function takes in a string and lowercases everything, normalizes unicode characters, and replaces
        anything that is not a letter, number, whits space, or single quote. It returns the cleaned string.'''
    string = string.lower()
    string = unicodedata.normalize('NFKD', string).encode('ascii', 'ignore').decode('utf-8')
    string = re.sub(r'[^0-9a-z/\'\s]', '', string)
    return string

In [7]:
df = pd.read_csv('news_inshorts.csv', index_col=[0])

In [19]:
df = df.reset_index().drop(columns='index')

In [20]:
df.title[1]

'Mohammad Shami confirms he is going to Australia ahead of T20 WC, shares pics from flight'

In [21]:
basic_clean(df.title[1])

'mohammad shami confirms he is going to australia ahead of t20 wc shares pics from flight'